In [1]:
import pandas as pd

In [2]:
sch_data = 'SCH-ZZ-20231208_035117.xlsx'      #Importing schedule data
inv_data = 'INV-ZZ-20231208_041852.xlsx'      #Importing inventory data

In [3]:
sch = pd.read_excel(sch_data)
inv = pd.read_excel(inv_data)

In [4]:
df1 = pd.DataFrame(sch)         #Converting excel data into dataframe
df2 = pd.DataFrame(inv) 

In [5]:
df2['Bool'] = 1   #Creating a new column in df2 and naming it as Bool and assigning all values as 1
                  #Bool value will be 0 for the flights which are cancelled

In [27]:
import numpy as np
import random
#Considering Aircraft Type Change

a = random.randrange(len(df2))
print(a)
cancelled_flights = [a]

293


In [28]:
df2.loc[a, ['AircraftType', 'TotalCapacity']]

AircraftType     Boeing 787
TotalCapacity           270
Name: 293, dtype: object

In [29]:
b = random.randrange(len(df2))
print(b)
df2.loc[b, ['AircraftType', 'TotalCapacity']]

1243


AircraftType     Airbus A320-200
TotalCapacity                150
Name: 1243, dtype: object

In [ ]:
#The AircraftType of the 1st flight is changed with that of the 2nd flight

In [30]:
#Number of affected passengers
num_affected_pax = df2.loc[a, 'TotalCapacity'] - df2.loc[b, 'TotalCapacity']

In [31]:
print(num_affected_pax)

120


In [32]:
for i in cancelled_flights:
    df2.at[i, 'Bool'] = 0     #Assigning a bool value of 0 to the cancelled flights
print(cancelled_flights)

[293]


In [33]:
threshold1 = pd.Timedelta(hours=6)
threshold2 = pd.Timedelta(hours=12)
threshold3 = pd.Timedelta(hours=24)
threshold4 = pd.Timedelta(hours=48)
threshold5 = pd.Timedelta(hours=72)
zero_time = pd.Timedelta(hours=0)
inf_time = pd.Timedelta(weeks=52)

In [34]:
#Algorithm for flight scoring based on arrival, departure time, arrival, departure time etc. 
for j in range(len(cancelled_flights)):
    new_col = f'Flight_Score_{cancelled_flights[j]}'
    new_col2 = f'Quality Score Grade_{cancelled_flights[j]}'
    df2[new_col] = 0                 #New column in the dataframe for scoring cancelled flights
    df2[new_col2] = 'NaN'            #Each flights having a bool value of 1 will have a score 
                                     #corresponding to each cancelled flight
    
    for i in range(len(df2)):
        if df2.loc[i, 'Bool'] == 1:  #Only scoring non-cancelled flights
            if df2.loc[i, 'DepartureAirport'] == df2.loc[cancelled_flights[j], 
            'DepartureAirport'] and df2.loc[i, 'ArrivalAirport'] == df2.loc[cancelled_flights[j], 'ArrivalAirport']:
               df2.loc[i, new_col] = 40 
               diff_dep = df2.loc[i, 'DepartureDateTime'] - df2.loc[cancelled_flights[j], 'DepartureDateTime']
               diff_arr = df2.loc[i, 'ArrivalDateTime'] - df2.loc[cancelled_flights[j], 'ArrivalDateTime']
               if((diff_dep <= threshold4) and (diff_dep >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                  
                    if((diff_dep <= threshold3) and (diff_dep >= zero_time)):                       
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10   #Adding only 10 because there is a nested-if 
                        if((diff_dep <= threshold2) and (diff_dep >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_dep <= threshold1) and (diff_dep >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               if((diff_arr <= threshold4) and (diff_arr >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                    
                    if((diff_arr <= threshold3) and (diff_arr >= zero_time)):                        
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                        if((diff_arr <= threshold2) and (diff_arr >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_arr <= threshold1) and (diff_arr >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               for k in range(len(df1)):
                    if (df2.loc[i, 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        tail_number = df1.loc[k, 'AircraftTailNumber']
               for k in range(len(df1)):
                    if (df2.loc[cancelled_flights[j], 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        if (df1.loc[k, 'AircraftTailNumber'] == tail_number):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 50

            #Quality grading of flights acc. to ruleset provided
            if(df2.loc[i, new_col] < 150 and df2.loc[i, new_col] > 0):
                df2.loc[i, new_col2] = 'D'
            if(df2.loc[i, new_col] >= 150):
                df2.loc[i, new_col2] = 'C'
            if(df2.loc[i, new_col] >= 180):
                df2.loc[i, new_col2] = 'B'
            if(df2.loc[i, new_col] >= 200):
                df2.loc[i, new_col2] = 'A'

df2 = pd.concat([df2], axis=1)

In [35]:
filt1 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'A')
filt2 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'B')
filt3 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'C')
filt4 = (df2['Oversold'] < 0)       #Considering only those flights which have vacant seats
Available_flights = df2.loc[(filt1 | filt2 | filt3) & filt4]

In [36]:
Available_flights

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,Bool,Flight_Score_293,Quality Score Grade_293
294,INV-ZZ-1050351,SCH-ZZ-1209990,ZZ,ZZ20240511BOMMAA3355.0,3355,Boeing 787,2024-05-11,2024-05-11 10:32:00,2024-05-11 13:38:00,BOM,...,63,45,56,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ...",1,170,C
295,INV-ZZ-4488406,SCH-ZZ-1209990,ZZ,ZZ20240512BOMMAA3355.0,3355,Boeing 787,2024-05-12,2024-05-12 10:32:00,2024-05-12 13:38:00,BOM,...,64,44,65,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ...",1,150,C


In [37]:
F = []          #Array for storing score for each alternate flight
Av_Fl = []      #Array for storing index of each alternate flight in the inventory dataframe
for i in range(len(Available_flights)):
    ix = Available_flights.index[i]
    Av_Fl.append(ix)
    F.append(df2.loc[ix, f'Quality Score Grade_{cancelled_flights[0]}'])     
    #Change the index based on the index of the flight cancelled

#Converting Quality Score into numerical scores
for i in range(len(F)):
    if F[i] == 'A':
        F[i] = 4     
    if F[i] == 'B':
        F[i] = 3
    if F[i] == 'C':
        F[i] = 2
    if F[i] == 'D': 
        F[i] = 1
        
V_S = []       #Array for storing number of available seats in each alternating flight
for i in range(len(Available_flights)):
    ix = Available_flights.index[i]
    V_S.append(-df2.loc[ix, 'Oversold'])
print(F)
print(Av_Fl)
print(V_S)

[2, 2]
[294, 295]
[99, 118]


In [38]:
#Scaling down the values in V_S 
for i in range(len(V_S)):
    V_S[i] = V_S[i]/10
    V_S[i] = np.floor(V_S[i])   #Floor value so that no. of passengers accommodated
print(V_S)                      #in each flight is less than the flight's capacity

[9.0, 11.0]


In [58]:
PNR = 'PNRB-ZZ-20231208_062017.xlsx'     #Importing the PNR Data
Pass = 'PNRP-ZZ-20231208_111136.xlsx'    #Importing the Passenger Data

In [59]:
PNR1 = pd.read_excel(PNR)          
Pass1 = pd.read_excel(Pass)

In [60]:
PNR_DF = pd.DataFrame(PNR1)             #Converting excel data into dataframe
Pass_DF = pd.DataFrame(Pass1)

In [61]:
Pass_DF['SSR_CODE_CD1'] = Pass_DF['SSR_CODE_CD1'].fillna(0)
Pass_DF['TierLevel'] = Pass_DF['TierLevel'].fillna(0)

In [62]:
departure_list = PNR_DF['ORIG_CD'].to_list()
arrival_list = PNR_DF['DEST_CD'].to_list()

city_pairs = [tuple([departure_list[i], arrival_list[i]]) for i in range(len(departure_list))]

fltnum_list = PNR_DF["FLT_NUM"].to_list()

citypair_dict = {fltnum_list[i] : city_pairs[i] for i in range(len(city_pairs))}

In [63]:
paxcnt_wt = 50
connect_wt = 100
grp_threshold = 7
grp_wt = 500
class_wt = {
    'FirstClass' : 2000, 'BusinessClass' : 1750, 
    'PremiumEconomyClass' : 1500, 'EconomyClass' : 1250
}
# The airlines can vary the weights as per their rules set and have the choice to disregard certain factors as per their choice.

In [64]:
def get_recloc_score(df):
    recloc_sclist = []
    recloc_score = 0
    for i in range(len(df['RECLOC'])):
        recloc_score += df['PAX_CNT'][i]*paxcnt_wt + class_wt[df['COS_CD'][i]]
        if df['PAX_CNT'][i] >= grp_threshold:
            recloc_score += grp_wt
        recloc_sclist.append(recloc_score)
        recloc_score = 0

    df['RECLOC_SCORE'] = recloc_sclist

    return df    

#Due to multiple passengers having the same recloc, We have split calculating passenger 
#score into two parts : recloc_Score and passenger_score. The recloc score will initialise
#the pax_score variable and build up from there.

In [65]:
get_recloc_score(PNR_DF)

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,COS_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ,RECLOC_SCORE
0,DRGS80,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,3,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2150
1,YEZQ47,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,4,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2200
2,JDDM40,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,3,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
3,AVFF16,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,1,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2050
4,DBLE79,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78179,CJEL38,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,7,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2100
78180,NDXM23,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,1,9,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2200
78181,RIUQ17,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,5,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1500
78182,QLFS50,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,2,1,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1300


In [66]:
recloc_dict = {PNR_DF['RECLOC'][i] : PNR_DF['RECLOC_SCORE'][i] for i in range(len(PNR_DF['RECLOC']))}

In [67]:
loyalty_dict = {'Platinum' : 1800, 'Gold' : 1600, 'Silver' : 1500, 0 : 0}
ssr_wt = 200

In [68]:
def get_pnr_score(df):
    pnr_sclist = []
    pnr_score = 0
    for i in range(len(df['RECLOC'])):
        pnr_score += recloc_dict[df['RECLOC'][i]] + loyalty_dict[df['TierLevel'][i]]
        if df['SSR_CODE_CD1'][i] != 0:
            pnr_score += ssr_wt
        
        pnr_sclist.append(pnr_score)
        pnr_score = 0

    df['PNR_SCORE'] = pnr_sclist

    return df   

In [69]:
get_pnr_score(Pass_DF)

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE
0,DRGS80,2024-03-02 16:35:00,VNQ9748,Mitchell,Christina,CA,2003-06-01,1-769-506-8397,christinamitchell1@example.com,Driving License,299196000000,NaN,0,ADT,4.276771e+09,Gold,3750
1,DRGS80,2024-03-02 16:35:00,BAP1926,Parker,Rebecca,CA,2015-08-18,1-947-699-5143,rebeccaparker2@example.com,Passport,11901984430,NaN,0,CHD,8.493470e+09,Platinum,3950
2,DRGS80,2024-03-02 16:35:00,ALV9938,Swamy,Vardaniya,IN,1945-02-23,6102977873,vardaniyaswamy3@example.com,Govt-Id,4283733463,NaN,0,S65,6.731611e+09,Platinum,3950
3,YEZQ47,2024-03-02 16:35:00,CEU7075,Edwin,Stuvan,IN,2003-06-18,5791967032,stuvanedwin1@example.com,Passport,139267560,NaN,0,ADT,7.214819e+09,Platinum,4000
4,YEZQ47,2024-03-02 16:35:00,WFG9647,Hernandez,Andrea,CA,1969-07-03,+1 (378) 291-0325,andreahernandez2@example.com,Govt-Id,5459762750,NaN,0,ADT,NaN,0,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152508,RIUQ17,2023-08-20 13:28:00,NQF8798,Shankar,Divit,IN,1981-08-22,(347) 703-8971,divitshankar3@example.com,Driving License,169860000000,NaN,0,ADT,NaN,0,1500
152509,RIUQ17,2023-08-20 13:28:00,RJI4602,Henderson,Autumn,US,2013-09-14,(371) 218-4438,autumnhenderson4@example.com,Govt-Id,7100653216,NaN,0,CHD,5.882689e+09,Gold,3100
152510,RIUQ17,2023-08-20 13:28:00,VPC0920,Ramanathan,Hridaan,IN,1944-09-06,(437) 839-4309,hridaanramanathan5@example.com,Driving License,715385534,NaN,0,S65,NaN,0,1500
152511,QLFS50,2023-08-20 13:28:00,BIB2224,Baria,Taimur,IN,2010-06-19,(771) 284-3525,taimurbaria1@example.com,Govt-Id,4431751697,NaN,0,CHD,3.701390e+09,Gold,2900


In [70]:
#Function that returns the PNRs that are affected based on flight no., and departure date of the cancelled flight
def Affected_PNR(FLT_NUM, DEP_DATE, PNR_DF):
    AffectedPNR = PNR_DF.loc[(PNR_DF['FLT_NUM'] == FLT_NUM) & (PNR_DF['DEP_DT']==DEP_DATE)]
    return AffectedPNR

In [71]:
#Dataframe consisting of affected PNRs
aff = Affected_PNR(df2.loc[cancelled_flights[0],'FlightNumber'], df2.loc[cancelled_flights[0],'DepartureDate'], PNR_DF)

In [72]:
#Dataframe consisting of affected passengers
total = pd.merge(Pass_DF, aff[['RECLOC']], on='RECLOC', how='inner')

In [73]:
len(total)

171

In [ ]:
#num_affected_pax no. of affected passengers need to be identified 
#We will do this based on PNR score assigned

In [127]:
#Importing class from PNR Data to Passenger Data
aff_pax['Class'] = 'NaN'

for i in range(len(aff)):
    recloc_aff = aff.loc[aff.index[i], 'RECLOC']
    class_aff = aff.loc[aff.index[i], 'COS_CD']

    aff_pax.loc[aff_pax['RECLOC'] == recloc_aff, 'Class'] = class_aff

In [75]:
total = total.sort_values(by='PNR_SCORE', ascending=True)   #Sorting passengers from low to high PNR Score

In [87]:
aff_pax = pd.DataFrame(total.head(num_affected_pax))

In [88]:
aff_pax

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE
20,SNGP60,2023-12-10 12:29:00,GNG7504,Day,Mary,CA,2014-02-10,(702) 282-5040,maryday2@example.com,Govt-Id,197370551,NaN,0,CHD,NaN,0,1350
167,MHHC93,2024-02-23 13:58:00,YNG8686,Shenoy,Ritvik,IN,1938-06-13,(517) 600-3057,ritvikshenoy2@example.com,Driving License,952259000000,NRPS,0,S65,NaN,0,1350
161,HMXJ76,2024-05-01 12:25:00,WOR5921,Bhatt,Eva,IN,2006-01-08,(692) 504-1512,evabhatt2@example.com,Passport,71805514,NRPS,0,CHD,NaN,0,1350
23,SRHC34,2024-03-04 03:48:00,IVG6789,Roy,Shanaya,IN,1990-04-14,(693) 203-3752,shanayaroy3@example.com,Govt-Id,2625592548,NaN,0,ADT,NaN,0,1400
22,SRHC34,2024-03-04 03:48:00,ETD8927,Kaur,Divyansh,IN,1938-01-13,(318) 649-6156,divyanshkaur2@example.com,Passport,552492834,NRPS,0,S65,NaN,0,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NFBD20,2024-03-20 13:17:00,THG9336,Cruz,Julie,US,1969-03-13,(597) 132-8641,juliecruz4@example.com,Govt-Id,3557821236,NaN,0,ADT,9.181475e+09,Platinum,3500
100,PALI14,2023-10-28 09:27:00,HMF5151,Graham,Alison,CA,1997-07-14,(629) 299-4682,alisongraham2@example.com,Driving License,124907000000,NaN,0,ADT,4.924927e+09,Gold,3500
50,PSCS86,2023-12-31 02:44:00,CVH5843,Mani,Oorja,IN,1947-04-04,(587) 210-6051,oorjamani3@example.com,Govt-Id,6239863332,NaN,0,S65,3.063365e+09,Gold,3500
48,PSCS86,2023-12-31 02:44:00,CWV5527,Petty,April,US,2022-11-01,(554) 996-8473,aprilpetty1@example.com,Passport,574608000000,NaN,0,INS,1.628692e+09,Gold,3500


In [83]:
len(aff_pax)

120

In [91]:
#Grouping passengers based on PNR_Score
for i in range(len(aff_pax)):
    score = aff_pax.loc[aff_pax.index[i], 'PNR_SCORE']
    
    if score >= 3500:
        aff_pax.loc[aff_pax.index[i], 'PNR_SCORE'] = 5
    elif score >= 3000:
        aff_pax.loc[aff_pax.index[i], 'PNR_SCORE'] = 4
    elif score >= 2500:
        aff_pax.loc[aff_pax.index[i], 'PNR_SCORE'] = 3
    elif score >= 2000:
        aff_pax.loc[aff_pax.index[i], 'PNR_SCORE'] = 2
    else:
        aff_pax.loc[aff_pax.index[i], 'PNR_SCORE'] = 1

In [92]:
aff_pax['PNR_SCORE'].value_counts()

PNR_SCORE
1    45
4    35
2    30
3     5
5     5
Name: count, dtype: int64

In [93]:
value_counts = aff_pax['PNR_SCORE'].value_counts()

In [94]:
# Convert the Series to a DataFrame
freq_df = pd.DataFrame({'PNR_SCORE': value_counts.index, 'Frequency': value_counts.values})

In [95]:
freq_df

,PNR_SCORE,Frequency
0,1,45
1,4,35
2,2,30
3,3,5
4,5,5


In [96]:
Score = []         #Array for storing PNR scores
G = []             #Array for storing number of passengers 
                   #corresponding to the PNR Score
for i in range(len(freq_df)):
    Score.append(freq_df.loc[i, 'PNR_SCORE'])
    G.append(freq_df.loc[i, 'Frequency'])
print(Score)
print(G)

[1, 4, 2, 3, 5]
[45, 35, 30, 5, 5]


In [97]:
#Scaling down the values in G
for i in range(len(G)):
    G[i] = G[i]/10
    G[i] = np.ceil(G[i])   #Ceil value so that number of passengers accommodated in
                           #a flight is always less than than the flight's capacity
print(G)

[5.0, 4.0, 3.0, 1.0, 1.0]


In [98]:
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import LinearInequalityToPenalty, QuadraticProgramToQubo

In [99]:
#Formulating the problem
qp = QuadraticProgram()

#Defining variables
for j in range(len(F)):
    for i in range(len(Score)):
        qp.binary_var(name=f'x_{i}_{j}')

#Objective function
linear_dict = {f'x_{i}_{j}': F[j] * Score[i] for i in range(len(Score)) for j in range(len(F))}
qp.maximize(linear=linear_dict)

#Add constraints
#Passenger Constraint (Each passenger(group here) is assigned one flight)
for k in range(len(Score)):
    constraint1 = {f'x_{k}_{j}': 1 for j in range(len(F))}
    qp.linear_constraint(linear=constraint1, sense='<=', rhs=1)

#Flight Constraint (Flights shouldn't accommodate more than it can carry/fly)
for k in range(len(F)):
    constraint2 = {f'x_{i}_{k}': G[i] for i in range(len(Score))}
    qp.linear_constraint(linear=constraint2, sense='<=', rhs=V_S[k])

print(qp.prettyprint())

Problem name: 

Maximize
  2*x_0_0 + 2*x_0_1 + 8*x_1_0 + 8*x_1_1 + 4*x_2_0 + 4*x_2_1 + 6*x_3_0 + 6*x_3_1
  + 10*x_4_0 + 10*x_4_1

Subject to
  Linear constraints (7)
    x_0_0 + x_0_1 <= 1  'c0'
    x_1_0 + x_1_1 <= 1  'c1'
    x_2_0 + x_2_1 <= 1  'c2'
    x_3_0 + x_3_1 <= 1  'c3'
    x_4_0 + x_4_1 <= 1  'c4'
    5*x_0_0 + 4*x_1_0 + 3*x_2_0 + x_3_0 + x_4_0 <= 9  'c5'
    5*x_0_1 + 4*x_1_1 + 3*x_2_1 + x_3_1 + x_4_1 <= 11  'c6'

  Binary variables (10)
    x_0_0 x_1_0 x_2_0 x_3_0 x_4_0 x_0_1 x_1_1 x_2_1 x_3_1 x_4_1



In [100]:
#Converting our problem to QUBO using optimization converters
ineq2pen_2 = QuadraticProgramToQubo()
qubo = ineq2pen_2.convert(qp)
print(qubo.prettyprint())
#All the constraints have been converted to penalty terms and added 
#to the objective function as well as slack variables are introduced

Problem name: 

Minimize
  366*c5@int_slack@0^2 + 1464*c5@int_slack@0*c5@int_slack@1
  + 2928*c5@int_slack@0*c5@int_slack@2 + 1464*c5@int_slack@0*c5@int_slack@3
  + 1464*c5@int_slack@1^2 + 5856*c5@int_slack@1*c5@int_slack@2
  + 2928*c5@int_slack@1*c5@int_slack@3 + 5856*c5@int_slack@2^2
  + 5856*c5@int_slack@2*c5@int_slack@3 + 1464*c5@int_slack@3^2
  + 366*c6@int_slack@0^2 + 1464*c6@int_slack@0*c6@int_slack@1
  + 2928*c6@int_slack@0*c6@int_slack@2 + 2928*c6@int_slack@0*c6@int_slack@3
  + 1464*c6@int_slack@1^2 + 5856*c6@int_slack@1*c6@int_slack@2
  + 5856*c6@int_slack@1*c6@int_slack@3 + 5856*c6@int_slack@2^2
  + 11712*c6@int_slack@2*c6@int_slack@3 + 5856*c6@int_slack@3^2
  + 3660*x_0_0*c5@int_slack@0 + 7320*x_0_0*c5@int_slack@1
  + 14640*x_0_0*c5@int_slack@2 + 7320*x_0_0*c5@int_slack@3 + 9150*x_0_0^2
  + 61*x_0_0*x_0_1 + 14640*x_0_0*x_1_0 + 10980*x_0_0*x_2_0 + 3660*x_0_0*x_3_0
  + 3660*x_0_0*x_4_0 + 3660*x_0_1*c6@int_slack@0 + 7320*x_0_1*c6@int_slack@1
  + 14640*x_0_1*c6@int_slack@2 + 14

In [101]:
#Converting our QUBO problem to Ising Model 
H, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("H:")
print(H)    #The Hamiltonian for our problem

offset: 18346.25
H:
SparsePauliOp(['IIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIZII', 'IIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIZIII', 'IIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIZIIII', 'IIIIIIIIZIIIIIIIII', 'IIIIIIIZIIIIIIIIII', 'IIIIIIZIIIIIIIIIII', 'IIIIIZIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIZIIIIZ', 'IIIIIIIZIIIIIIIIIZ', 'IIIIIIZIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIZ', 'IIIIZIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIZIIIIZI', 'IIIIIIIZIIIIIIIIZI', 'IIIIIIZIIIIIIIIIZI', 'IIIIIZIIIIIIIIIIZI', 'IIIIZIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIZIZII', 'IIIIIIIIIIZIIIIZII', 'IIIIIIIZIIIIIIIZII', 'IIIIIIZIIIIIIIIZII', 'IIIIIZIIIIIIIIIZII', 'IIIIZIIIIIIIIIIZII', 'IIIIIIIIIIIIIZZIII', 'IIIIIIIIIZIIIIZIII

In [102]:
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(),reps=1)

In [103]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

In [104]:
qubo_result = qaoa.solve(qubo)
print(qubo_result.prettyprint())

objective function value: -30.0
variable values: x_0_0=0.0, x_1_0=0.0, x_2_0=1.0, x_3_0=0.0, x_4_0=1.0, x_0_1=1.0, x_1_1=1.0, x_2_1=0.0, x_3_1=1.0, x_4_1=0.0, c5@int_slack@0=1.0, c5@int_slack@1=0.0, c5@int_slack@2=1.0, c5@int_slack@3=0.0, c6@int_slack@0=1.0, c6@int_slack@1=0.0, c6@int_slack@2=0.0, c6@int_slack@3=0.0
status: SUCCESS


In [105]:
print("variable order:", [var.name for var in qubo_result.variables])
X = []
for i in range(min(4, len(qubo_result.samples))):
    sample = qubo_result.samples[i]

    # Extract 'x' array from the sample
    x_array = sample.x
    X.append(x_array[:len(Score) * len(F)])   #Omitting the slack variables

variable order: ['x_0_0', 'x_1_0', 'x_2_0', 'x_3_0', 'x_4_0', 'x_0_1', 'x_1_1', 'x_2_1', 'x_3_1', 'x_4_1', 'c5@int_slack@0', 'c5@int_slack@1', 'c5@int_slack@2', 'c5@int_slack@3', 'c6@int_slack@0', 'c6@int_slack@1', 'c6@int_slack@2', 'c6@int_slack@3']


In [106]:
X #List that contains the first 4 optimal solutions

[array([0., 0., 1., 0., 1., 1., 1., 0., 1., 0.]),
 array([0., 0., 1., 1., 0., 1., 1., 0., 0., 1.]),
 array([0., 1., 0., 0., 0., 1., 0., 1., 1., 1.]),
 array([0., 0., 1., 1., 0., 1., 1., 0., 0., 1.])]

In [107]:
#Anslyzing first solution
n=1
X[n-1]

array([0., 0., 1., 0., 1., 1., 1., 0., 1., 0.])

In [108]:
aff_pax['Alt_Fli'] = 'NaN'      #New column for storing the assigned alternate flight to passengers

In [109]:
for i in range(len(X[n-1])):
    if X[n-1][i] == 1:
        alt_flight_number = (i // len(Score)) + 1
        group_index = (i % len(Score)) + 1
        assignment_info = f"Group {group_index} is assigned to Flight {alt_flight_number}" 
        print(assignment_info)
        condition = aff_pax['PNR_SCORE'] == Score[group_index - 1]
        aff_pax.loc[condition, 'Alt_Fli'] = Av_Fl[alt_flight_number - 1]       

Group 3 is assigned to Flight 1
Group 5 is assigned to Flight 1
Group 1 is assigned to Flight 2
Group 2 is assigned to Flight 2
Group 4 is assigned to Flight 2


In [110]:
aff_pax['Alt_Fli'].value_counts()  #No. of passengers in each flight

Alt_Fli
295    85
294    35
Name: count, dtype: int64

In [111]:
filt = (aff_pax['Alt_Fli'] != 'NaN')
Reaccommodated_List_df = aff_pax.loc[filt] #Dataframe that contains the passengers who were rescheduled

In [112]:
Reaccommodated_List_df.head()

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE,Alt_Fli
20,SNGP60,2023-12-10 12:29:00,GNG7504,Day,Mary,CA,2014-02-10,(702) 282-5040,maryday2@example.com,Govt-Id,197370551,NaN,0,CHD,NaN,0,1,295
167,MHHC93,2024-02-23 13:58:00,YNG8686,Shenoy,Ritvik,IN,1938-06-13,(517) 600-3057,ritvikshenoy2@example.com,Driving License,952259000000,NRPS,0,S65,NaN,0,1,295
161,HMXJ76,2024-05-01 12:25:00,WOR5921,Bhatt,Eva,IN,2006-01-08,(692) 504-1512,evabhatt2@example.com,Passport,71805514,NRPS,0,CHD,NaN,0,1,295
23,SRHC34,2024-03-04 03:48:00,IVG6789,Roy,Shanaya,IN,1990-04-14,(693) 203-3752,shanayaroy3@example.com,Govt-Id,2625592548,NaN,0,ADT,NaN,0,1,295
22,SRHC34,2024-03-04 03:48:00,ETD8927,Kaur,Divyansh,IN,1938-01-13,(318) 649-6156,divyanshkaur2@example.com,Passport,552492834,NRPS,0,S65,NaN,0,1,295


In [113]:
filt = (aff_pax['Alt_Fli'] == 'NaN')
Exception_List_df = aff_pax.loc[filt] #Dataframe that contains the passengers who couldn't be rescheduled

In [114]:
Exception_List_df.head()

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE,Alt_Fli


In [115]:
print(f'{len(Reaccommodated_List_df)} passengers out of {len(aff_pax)} were reaccommodated')

120 passengers out of 120 were reaccommodated


In [ ]:
#Importing class from PNR Data to Passenger Data
aff_pax['Class'] = 'NaN'

for i in range(len(aff)):
    recloc_aff = aff.loc[aff.index[i], 'RECLOC']
    class_aff = aff.loc[aff.index[i], 'COS_CD']

    aff_pax.loc[aff_pax['RECLOC'] == recloc_aff, 'Class'] = class_aff

In [116]:
aff_pax['Assigned Class'] = ''     #New column for storing the newly assigned class in alternate flights

In [132]:
#Alternate flight 1
f = 1
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered = aff_pax.loc[filt]

In [133]:
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered_pass_data_1 = aff_pax.loc[filt]

In [134]:
# Assign classes based on 'Score' and class limits
def assign_class(row):
     global class_count
     for category, limit in class_limits.items():
           if class_count[category] < limit:
               class_count[category] += 1
               return category
     return None

In [135]:
#Assigning class based on PNR scores
filt_sorted_1 = filtered_pass_data_1.copy()  # Making a copy to avoid modifying the original DataFrame

# Sort the DataFrame based on 'Score' in descending order
filt_sorted_1 = filt_sorted_1.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {'FirstClass': df2.loc[Av_Fl[f-1], 'FC_Oversold'], 'BusinessClass': df2.loc[Av_Fl[f-1], 'BC_Oversold'], 'PremiumEconomyClass': df2.loc[Av_Fl[f-1], 'PC_Oversold'], 'EconomyClass': df2.loc[Av_Fl[f-1], 'EC_Oversold']} 

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filt_sorted_1['Assigned Class'] = filt_sorted_1.apply(assign_class, axis=1)

In [136]:
#Alternate flight 2
f = 2
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered = aff_pax.loc[filt]

In [137]:
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered_pass_data_2 = aff_pax.loc[filt]

In [138]:
filt_sorted_2 = filtered_pass_data_2.copy()  # Making a copy to avoid modifying the original DataFrame

# Sort the DataFrame based on 'Score' in descending order
filt_sorted_2 = filt_sorted_2.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {'FirstClass': df2.loc[Av_Fl[f-1], 'FC_Oversold'], 'BusinessClass': df2.loc[Av_Fl[f-1], 'BC_Oversold'], 'PremiumEconomyClass': df2.loc[Av_Fl[f-1], 'PC_Oversold'], 'EconomyClass': df2.loc[Av_Fl[f-1], 'EC_Oversold']}  

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filt_sorted_2['Assigned Class'] = filt_sorted_2.apply(assign_class, axis=1)

In [139]:
#Passengers who were alloted the same class in flight 1
filt = (filt_sorted_1['Class'] == filt_sorted_1['Assigned Class'])
same_allotment_1 = filt_sorted_1.loc[filt]   

In [140]:
#Passengers who were alloted the same class in flight 2
filt = (filt_sorted_2['Class'] == filt_sorted_2['Assigned Class'])
same_allotment_2 = filt_sorted_2.loc[filt]

In [141]:
#Passengers who were downgraded 

In [142]:
#Flight 1

In [143]:
filt1 = (filt_sorted_1['Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
FC_dn1 = filt_sorted_1.loc[filt1 & (filt2 | filt3| filt4)]

In [144]:
filt2 = (filt_sorted_1['Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
BC_dn1 = filt_sorted_1.loc[filt2 & (filt3 | filt4)]

In [145]:
filt3 = (filt_sorted_1['Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
PC_dn1 = filt_sorted_1.loc[filt3 & (filt4)]

In [146]:
#Flight 2

In [147]:
filt1 = (filt_sorted_2['Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
FC_dn2 = filt_sorted_2.loc[filt1 & (filt2 | filt3| filt4)]

In [148]:
filt2 = (filt_sorted_2['Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
BC_dn2 = filt_sorted_2.loc[filt2 & (filt3 | filt4)]

In [149]:
filt3 = (filt_sorted_2['Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
PC_dn2 = filt_sorted_2.loc[filt3 & (filt4)]

In [150]:
#Passengers who were upgraded 

In [151]:
#Flight 1

In [152]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Class'] == 'EconomyClass')
EC_up1 = filt_sorted_1.loc[filt4 & (filt1 | filt2| filt3)]

In [153]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Class'] == 'PremiumEconomyClass')
PC_up1 = filt_sorted_1.loc[filt3 & (filt1 | filt2)]

In [154]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Class'] == 'BusinessClass')
BC_up1 = filt_sorted_1.loc[filt2 & filt1]

In [155]:
#Flight 2

In [156]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Class'] == 'EconomyClass')
EC_up2 = filt_sorted_2.loc[filt4 & (filt1 | filt2| filt3)]

In [157]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Class'] == 'PremiumEconomyClass')
PC_up2 = filt_sorted_2.loc[filt3 & (filt1 | filt2)]

In [158]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Class'] == 'BusinessClass')
BC_up2 = filt_sorted_2.loc[filt2 & filt1]

In [159]:
print(f"{len(same_allotment_1) + len(same_allotment_2)} passengers were allotted the same class they applied for")
print(f"{len(FC_dn1) + len(BC_dn1) + len(PC_dn1) + len(FC_dn2) + len(BC_dn2) + len(PC_dn2)} passengers were downgraded")
print(f"{len(EC_up1) + len(PC_up1) + len(BC_up1) + len(EC_up2) + len(PC_up2) + len(BC_up2)} passengers were upgraded")

22 passengers were allotted the same class they applied for
27 passengers were downgraded
71 passengers were upgraded
